In [1]:
import mesa
import numpy as np
from mesa.visualization import SolaraViz,make_plot_measure

<IPython.core.display.Javascript object>

In [2]:
class WealthAgent(mesa.Agent):
    
    def __init__(self,model, proportion):
        super().__init__(model)
        self.wealth=10
        self.W =proportion
        
        
    def step(self):
        #increase welath by proportion - payday
        self.wealth += (self.W*self.wealth)
        #self.wealth -= 10000 #surival expense
        
        if self.wealth > 0: 
            #get basic expenses
            exchange_agent = self.random.choice(self.model.agents)
            if exchange_agent is not None and exchange_agent is not self:
                #print(self.wealth)
                exchange_agent.wealth += (exchange_agent.W*self.wealth)
                self.wealth -= (exchange_agent.W*self.wealth)                
                

In [12]:
import solara
from matplotlib.figure import Figure

@solara.component
def Histogram(model):
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    wealth_vals = [agent.wealth for agent in model.agents]
    # Note: you have to use Matplotlib's OOP API instead of plt.hist
    # because plt.hist is not thread-safe.
    ax.hist(wealth_vals, bins=10)
    return solara.FigureMatplotlib(fig)

def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.agents]
    x = sorted(agent_wealths)
    N = model.population
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B

In [24]:
class WealthModel(mesa.Model): 
    
    def __init__(self, population, tax=False, debt=False):
                 #average_income = 38000,
                 #standard_deviation = 1818): #obtained from https://apps.bea.gov/scb/issues/2021/01-january/0121-revisions-to-gdp-gdi.htm
        
        super().__init__()
        self.population = population
        self.tax = tax
        self.tax_dynamic = int(tax*population)
        self.debt = debt
        
        #self.schedule = mesa.time.RandomActivation(self)
        self.datacollector = mesa.DataCollector(model_reporters = {"Gini": compute_gini },
                                               agent_reporters={"Wealth":"wealth"})
        
        
        # create an array of iniaital weatth value    
        gaussian_array = np.random.normal(loc=0.5,
                                          scale=0.15,
                                          size=self.population)
        # round array to two decimals
        gaussian_array = np.around(gaussian_array, decimals=2)
        
        for idx in range(self.population):
            WealthAgent(self, float(gaussian_array[idx]))
    
    def step(self):
        self.datacollector.collect(self)
        Histogram(self)
        self.agents.shuffle_do("step")
        # Tax
        if model.tax != None:
            # Sort agents from richest to poorest
            sorted_agents = sorted(self.agents, key=lambda agent: agent.wealth, reverse=True)
            for agent_idx in range(self.tax_dynamic): 
                #get percent
                tax_amount = sorted_agents[agent_idx].wealth*self.tax
                #tax wealthy
                sorted_agents[agent_idx].wealth -= tax_amount
                #give poor
                sorted_agents[-agent_idx].wealth += tax_amount
                
            
        

In [31]:
model = WealthModel(200, tax=0.5)

for step in range(10):
    model.step()
    print(step)
    
output = model.datacollector.get_agent_vars_dataframe()
output.to_csv("inequality_output.csv")

0
1
2
3
4
5
6
7
8
9


In [ ]:
model = WealthModel(50)

model_params = {
    "population": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of agents:",
        "min": 10,
        "max": 200,
        "step": 1,
    }
}

wealth_plot = make_plot_measure("Gini")

dash = SolaraViz(
    model, 
    components=[wealth_plot, Histogram],
    model_params=model_params,
)

dash